In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score

from sklearn import metrics
from collections import Counter

In [11]:
train_data = pd.read_csv("/kaggle/input/fraud-detection/fraudTrain.csv", index_col=0)

In [12]:
train_data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [13]:
train_data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [14]:
subset_data = train_data.copy()

In [19]:
import datetime
subset_data = subset_data[["trans_date_trans_time", "dob", "amt", "city_pop", "is_fraud"]]
#subset_data["trans_date"] = datetime.datetime(subset_data["trans_date_trans_time"])
subset_data.head()

,trans_date_trans_time,dob,amt,city_pop,is_fraud
0,2019-01-01 00:00:18,1988-03-09,4.97,3495,0
1,2019-01-01 00:00:44,1978-06-21,107.23,149,0
2,2019-01-01 00:00:51,1962-01-19,220.11,4154,0
3,2019-01-01 00:01:16,1967-01-12,45.00,1939,0
4,2019-01-01 00:03:06,1986-03-28,41.96,99,0


In [22]:
subset_data.dtypes

trans_date_trans_time     object
dob                       object
amt                      float64
city_pop                   int64
is_fraud                   int64
dtype: object

In [29]:
subset_data["trans_date"] = pd.to_datetime(pd.to_datetime(subset_data["trans_date_trans_time"], format="%Y-%m-%d %H:%M:%S").dt.date, format="%Y-%m-%d")
subset_data["dob_date"] = pd.to_datetime(subset_data["dob"], format="%Y-%m-%d")
subset_data.head()

,trans_date_trans_time,dob,amt,city_pop,is_fraud,trans_date,dob_date
0,2019-01-01 00:00:18,1988-03-09,4.97,3495,0,2019-01-01,1988-03-09
1,2019-01-01 00:00:44,1978-06-21,107.23,149,0,2019-01-01,1978-06-21
2,2019-01-01 00:00:51,1962-01-19,220.11,4154,0,2019-01-01,1962-01-19
3,2019-01-01 00:01:16,1967-01-12,45.00,1939,0,2019-01-01,1967-01-12
4,2019-01-01 00:03:06,1986-03-28,41.96,99,0,2019-01-01,1986-03-28


In [30]:
subset_data.dtypes

trans_date_trans_time            object
dob                              object
amt                             float64
city_pop                          int64
is_fraud                          int64
trans_date               datetime64[ns]
dob_date                 datetime64[ns]
dtype: object

In [33]:
subset_data["age"] = (subset_data["trans_date"]-subset_data["dob_date"]) / (np.timedelta64(1, 'D')*365)
subset_data.head()

,trans_date_trans_time,dob,amt,city_pop,is_fraud,trans_date,dob_date,age
0,2019-01-01 00:00:18,1988-03-09,4.97,3495,0,2019-01-01,1988-03-09,30.835616
1,2019-01-01 00:00:44,1978-06-21,107.23,149,0,2019-01-01,1978-06-21,40.558904
2,2019-01-01 00:00:51,1962-01-19,220.11,4154,0,2019-01-01,1962-01-19,56.989041
3,2019-01-01 00:01:16,1967-01-12,45.00,1939,0,2019-01-01,1967-01-12,52.005479
4,2019-01-01 00:03:06,1986-03-28,41.96,99,0,2019-01-01,1986-03-28,32.786301


In [48]:
import numpy as np
import pandas as pd
import math

def f1_score(data, y, mask, total_frd, min_recall):
	'''
	It returns the Information Gain of a variable given a loss function.
	y: target variable.
	mask: split choice.
	total_frd: Total Fraud that can be captured in the starting population
	min_recall: Minimum recall set to be achieved
	'''
	
	hit_data = data[mask]
	
	capture = hit_data['fraud_amount'].sum()
	temp_tot_frd = data['fraud_amount'].sum()
	hit_rate = capture/hit_data['transaction_amount'].sum()
	capture_rate = capture/temp_tot_frd

	f1 = (2*hit_rate*capture_rate)/(hit_rate+capture_rate)

	if f1 is None:
		return 0

	if capture_rate < min_recall:
		return 0

	return f1

def max_f1_score_split(data, x, y, total_frd, subset_x, min_recall):
	'''
	Given a predictor & target variable, returns the best split, the error and the type of variable based on a selected cost function.
	x: predictor variable as Pandas Series.
	y: target variable as Pandas Series.
	total_frd: Total Fraud that can be captured in the starting population
	min_recall: Minimum recall set to be achieved
	'''

	split_value = []
	f1 = [] 
	le_gr = []
	print("Checking threshold for {}".format(x.name))
	options = subset_x.sort_values().unique()[1:]
	print("Old size was {}".format(len(options)))
	subset_x = subset_x[subset_x >= 0]

	options = subset_x.sort_values().unique()[1:]

	if len(options) > 100:
		arr_percentiles = np.arange(0,100)
		final_options = np.zeros(100)
		np.percentile(options, arr_percentiles, out = final_options)
	else:
		final_options = options
	print("Current size is {}".format(len(final_options)))

	# Calculate ig for all values
	for ind, val in enumerate(options):
		nonull_data = data[x>-9998]
		mask =   x < val
		val_f1 = f1_score(nonull_data, y, mask, total_frd, min_recall)
		mask_2 = x > val
		val_new_f1 = f1_score(nonull_data, y, mask_2, total_frd, min_recall)
		# Append results
		if val_new_f1 < val_f1:
			le_gr.append(1)
			f1.append(val_f1)
		else:
			le_gr.append(0)
			f1.append(val_new_f1)
		split_value.append(val)

	# Check if there are more than 1 results if not, return False
	if len(f1) == 0:
		return(None,None,None, False)

	else:
	# Get results with highest IG
		best_f1 = max(f1)
		best_f1_index = f1.index(best_f1)
		best_split = split_value[best_f1_index]
		best_ineq = le_gr[best_f1_index]
		return(best_f1,best_split,best_ineq, True)

def get_best_split(y, data, x_vars, total_frd, min_recall):
	'''
	Given a data, select the best split and return the variable, the value, the variable type and the information gain.
	y: name of the target variable
	data: dataframe where to find the best split.
	x_vars: The variables used for decision making
	total_frd: Total fraud that can be captured in the starting population
	min_recall: Minimum recall set to be achieved
	'''

	split_value = []
	f1 = []
	le_gr = []

	for x in x_vars:
		dropped_data = data.dropna(axis=0, subset = [x])
		subset_data = dropped_data[dropped_data[y]==1]
		f1_score, split, ineq, _ = max_f1_score_split(dropped_data, dropped_data[x], dropped_data[y], total_frd, subset_data[x], min_recall)
		if f1_score is None:
			print("Found None")
			f1_score = 0
		le_gr.append(ineq)
		f1.append(f1_score)
		split_value.append(split)

	best_f1 = max(f1)
	best_f1_index = f1.index(best_f1)
	best_split = split_value[best_f1_index]
	best_ineq = le_gr[best_f1_index]
	best_var = x_vars[best_f1_index]
	return(best_var, best_split, best_f1, best_ineq)

def make_split(variable, value, data, ineq):
	'''
	Given a data and a split conditions, do the split.
	variable: variable with which make the split.
	value: value of the variable to make the split.
	data: data to be splitted.
	ineq: Greater than or less than inequality
	'''
	print(variable)
	print(value)
	if(ineq==1):
		data_1 = data[data[variable] < value]
	else:
		data_1 = data[data[variable] > value]

	return data_1

def calc_metrics(data, total_frd):
	'''
	Given the target variable, make a prediction.
	data: pandas series for target variable
	total_frd: Total Fraud that can be captured in the starting population
	'''

	hit_data = data

	capture = hit_data['fraud_amount'].sum()
	hit_rate = capture/hit_data['transaction_amount'].sum()
	capture_rate = capture/total_frd

	return hit_rate, capture_rate

def train_tree(data,y, total_frd, x_vars, max_depth = None, min_samples_split = None, min_recall = None, min_recall_overall = None, min_precision = 0.1, counter = 0):
	'''
	Trains a Decission Tree
	data: Data to be used to train the Decission Tree
	y: target variable column name
	total_frd: Total fraud that can be captured in the starting population
	x_vars: The variables used for decision making
	max_depth: maximum depth to stop splitting.
	min_samples_split: minimum number of observations to make a split.
	min_recall: minimum recall for each threshold selection
	min_recall_overall: minimum recall for the overall rule
	min_precision: Stop once minimum precision reaches a certain level
	'''

	# check for depth conditions
	print(counter)
	precision, recall = calc_metrics(data, total_frd)

	if max_depth == None:
		depth_cond = True

	else:
		if counter < max_depth:
			depth_cond = True

		else:
			depth_cond = False

	# Check for sample conditions
	if min_samples_split == None:
		sample_cond = True

	else:
		if data.shape[0] > min_samples_split:
			sample_cond = True

		else:
			sample_cond = False

	# Check for recall condition
	if min_recall_overall == None:
		recall_cond = True

	else:
		if min_recall_overall < recall:
			recall_cond = True

		else:
			recall_cond = False

	# Check for condition
	if depth_cond & sample_cond & recall_cond:

		var, val, f1, ineq = get_best_split(y, data, x_vars, total_frd, min_recall)
		counter += 1

		new_data = make_split(var, val, data, ineq)

		# Instantiate sub-tree
		if ineq == 1:
			split_type = "<"
		else:
			split_type = ">"
		question = "{} {} {}".format(var, split_type, val)
		new_precision, new_recall = calc_metrics(new_data, total_frd)
		question = question + " Precision:{}, Recall:{}".format(new_precision, new_recall)
		path = [question]
		print(question)


		#Find answers (recursion)

		next_path = train_tree(new_data, y, total_frd, x_vars, max_depth, min_samples_split, min_recall, min_recall_overall, min_precision, counter)

		path.append(next_path)

		return path

	print("Run Complete")
	return None

In [43]:
algo_data = subset_data.copy()
algo_data["transaction_amount"] = algo_data["amt"]
algo_data["fraud_amount"] = np.where(algo_data["is_fraud"]==1, algo_data["amt"], 0)
algo_data = algo_data[["transaction_amount", "city_pop", "age", "fraud_amount", "is_fraud"]]
algo_data.head()

,transaction_amount,city_pop,age,fraud_amount,is_fraud
0,4.97,3495,30.835616,0.0,0
1,107.23,149,40.558904,0.0,0
2,220.11,4154,56.989041,0.0,0
3,45.00,1939,52.005479,0.0,0
4,41.96,99,32.786301,0.0,0


In [45]:
total_frd = algo_data[algo_data["is_fraud"]==1]["transaction_amount"].sum()
print(total_frd)

3988088.6100000003


In [44]:
print(algo_data["fraud_amount"].sum())

3988088.6100000003


In [50]:
algo_data.shape

(1296675, 5)

In [54]:
rf_vars = ["transaction_amount", "city_pop", "age"]
decisions = train_tree(algo_data,"is_fraud", total_frd, rf_vars, max_depth = 50, min_samples_split = None, min_recall = 0.9, min_recall_overall = 0.3, min_precision = None, counter = 0)

0
Checking threshold for transaction_amount
Old size was 6638
Current size is 100


/tmp/ipykernel_33/1007425240.py:21: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2*hit_rate*capture_rate)/(hit_rate+capture_rate)


Checking threshold for city_pop
Old size was 690
Current size is 100


/tmp/ipykernel_33/1007425240.py:18: RuntimeWarning: invalid value encountered in scalar divide
  hit_rate = capture/hit_data['transaction_amount'].sum()


Checking threshold for age
Old size was 1448
Current size is 100


...

/tmp/ipykernel_33/1007425240.py:18: RuntimeWarning: invalid value encountered in scalar divide
  hit_rate = capture/hit_data['transaction_amount'].sum()


Checking threshold for city_pop
Old size was 666
Current size is 100


/tmp/ipykernel_33/1007425240.py:18: RuntimeWarning: invalid value encountered in scalar divide
  hit_rate = capture/hit_data['transaction_amount'].sum()


Checking threshold for age
Old size was 1241
Current size is 100
transaction_amount
741.98
transaction_amount > 741.98 Precision:0.5238822504135147, Recall:0.6924773444289142
50
Run Complete


/tmp/ipykernel_33/1007425240.py:21: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2*hit_rate*capture_rate)/(hit_rate+capture_rate)


In [55]:
decisions

['transaction_amount > 288.3 Precision:0.17930064373603957, Recall:0.955395998585899',
 ['transaction_amount < 1206.74 Precision:0.230919172467846, Recall:0.9412035080133287',
  ['transaction_amount > 296.02 Precision:0.2321296623194591, Recall:0.9274303310928692',
   ['transaction_amount > 357.15 Precision:0.2403241921043301, Recall:0.834792419519485',
    ['transaction_amount > 728.77 Precision:0.5011109275735717, Recall:0.7516577496506528',
     ['transaction_amount < 1176.2 Precision:0.5098730800434289, Recall:0.7414814536931766',
      ['transaction_amount > 736.16 Precision:0.5146874821457248, Recall:0.7348703317803161',
       ['transaction_amount > 736.56 Precision:0.5148360128931962, Recall:0.7341318050603696',
        ['transaction_amount > 737.68 Precision:0.5152683983360974, Recall:0.732653412633176',
         ['transaction_amount > 738.68 Precision:0.5156366952868978, Recall:0.7311727810380824',
          ['transaction_amount > 738.78 Precision:0.5155774822950125, Recall:0